# API SQLite3 con Flask
Este notebook contiene las funciones para interactuar con SQLite3 y una API Flask para exponer los endpoints.

## 1. Funciones de Base de Datos

In [ ]:
from __future__ import annotations

import os
import re
import sqlite3
from typing import Any, Dict, List, Tuple, Optional

# ------------------------------
# Config & Pluggable Connection
# ------------------------------

# Environment variables (defaults for SQLite)
DB_BACKEND = os.getenv('DB_BACKEND', 'sqlite').lower()
SQLITE_PATH = os.getenv('DB_FILE_PATH', 'dev.db')

def get_db_connection(db_path: Optional[str] = None):
    """
    Connection factory. Replace/extend this to support other DB engines.
    Currently supports SQLite.
    """
    backend = DB_BACKEND

    if backend == 'sqlite':
        path = db_path or SQLITE_PATH
        return sqlite3.connect(path)
    else:
        raise NotImplementedError(f"DB_BACKEND '{backend}' not implemented yet.")

# ------------------------------
# Helpers
# ------------------------------

_IDENTIFIER_RX = re.compile(r'^[A-Za-z_][A-Za-z0-9_]*$')

def validate_identifier(name: str, what: str = 'identifier') -> str:
    """
    Basic whitelist validation for table/column identifiers to reduce SQL injection risk.
    """
    if not isinstance(name, str) or not _IDENTIFIER_RX.match(name):
        raise ValueError(f"Invalid {what}: {name!r}")
    return name

def rows_to_dicts(cursor, rows: List[Tuple]) -> List[Dict[str, Any]]:
    columns = [col[0] for col in cursor.description] if cursor.description else []
    result = []
    for row in rows:
        if columns:
            result.append({col: row[idx] for idx, col in enumerate(columns)})
        else:
            result.append({'value': row})
    return result

# ------------------------------
# Core DB Functions
# ------------------------------

def db_tables(data_db: str = SQLITE_PATH) -> List[str]:
    """
    Devuelve el listado de tablas de la base de datos.
    """
    conn = get_db_connection(data_db)
    try:
        cur = conn.cursor()
        cur.execute(
            "SELECT name FROM sqlite_master WHERE type = 'table' AND name NOT LIKE 'sqlite_%' ORDER BY name"
        )
        rows = cur.fetchall()
        return [r[0] for r in rows]
    finally:
        conn.close()

def db_table_schema(tabla: str, data_db: str = SQLITE_PATH) -> List[Dict[str, Any]]:
    """
    Devuelve la lista de campos (columnas) de una tabla.
    """
    t = validate_identifier(tabla, 'table name')
    conn = get_db_connection(data_db)
    try:
        cur = conn.cursor()
        cur.execute(f'PRAGMA table_info({t})')
        rows = cur.fetchall()
        return [
            {
                'cid': r[0],
                'name': r[1],
                'type': r[2],
                'notnull': bool(r[3]),
                'dflt_value': r[4],
                'pk': bool(r[5]),
            }
            for r in rows
        ]
    finally:
        conn.close()

def db_update(tabla: str, campo: str, valor: Any, condicion_sql: str, data_db: str = SQLITE_PATH) -> Dict[str, Any]:
    """
    UPDATE <tabla> SET <campo> = ? WHERE <condicion_sql>
    """
    t = validate_identifier(tabla, 'table name')
    c = validate_identifier(campo, 'column name')
    sql = f'UPDATE {t} SET {c} = ? WHERE {condicion_sql}'
    conn = get_db_connection(data_db)
    try:
        cur = conn.cursor()
        cur.execute(sql, (valor,))
        conn.commit()
        res = {'rowcount': cur.rowcount}
    finally:
        conn.close()
    return res

def db_insert(tabla: str, json_valores: Dict[str, Any], data_db: str = SQLITE_PATH) -> Dict[str, Any]:
    """
    INSERT INTO <tabla> (<cols...>) VALUES (<placeholders...>)
    """
    t = validate_identifier(tabla, 'table name')
    if not isinstance(json_valores, dict) or not json_valores:
        raise ValueError('json_valores debe ser un objeto con al menos una clave.')
    cols = [validate_identifier(k, 'column name') for k in json_valores.keys()]
    vals = list(json_valores.values())
    placeholders = ', '.join(['?'] * len(cols))
    col_list = ', '.join(cols)
    sql = f'INSERT INTO {t} ({col_list}) VALUES ({placeholders})'
    conn = get_db_connection(data_db)
    try:
        cur = conn.cursor()
        cur.execute(sql, vals)
        conn.commit()
        res = {'rowcount': cur.rowcount, 'lastrowid': getattr(cur, 'lastrowid', None)}
    finally:
        conn.close()
    return res

def db_delete(tabla: str, condicion_sql: str, data_db: str = SQLITE_PATH) -> Dict[str, Any]:
    """
    DELETE FROM <tabla> WHERE <condicion_sql>
    """
    t = validate_identifier(tabla, 'table name')
    sql = f'DELETE FROM {t} WHERE {condicion_sql}'
    conn = get_db_connection(data_db)
    try:
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
        res = {'rowcount': cur.rowcount}
    finally:
        conn.close()
    return res

def db_delete_pk(tabla: str, pk: str, valor: Any, data_db: str = SQLITE_PATH) -> Dict[str, Any]:
    """
    DELETE FROM <tabla> WHERE <pk> = ?
    """
    t = validate_identifier(tabla, 'table name')
    p = validate_identifier(pk, 'primary key column')
    sql = f'DELETE FROM {t} WHERE {p} = ?'
    conn = get_db_connection(data_db)
    try:
        cur = conn.cursor()
        cur.execute(sql, (valor,))
        conn.commit()
        res = {'rowcount': cur.rowcount}
    finally:
        conn.close()
    return res


## 2. Ejemplos de Uso de las Funciones

In [ ]:
# Crear una base de datos de ejemplo
import sqlite3

# Crear base de datos de prueba
conn = sqlite3.connect('dev.db')
cursor = conn.cursor()

# Crear tabla de usuarios
cursor.execute('''
    CREATE TABLE IF NOT EXISTS usuarios (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        edad INTEGER
    )
''')

# Insertar datos de ejemplo
cursor.execute('''
    INSERT OR IGNORE INTO usuarios (nombre, email, edad) 
    VALUES 
    ('Juan Pérez', 'juan@example.com', 30),
    ('María García', 'maria@example.com', 25),
    ('Carlos López', 'carlos@example.com', 35)
''')

conn.commit()
conn.close()

print('Base de datos de ejemplo creada exitosamente!')


In [ ]:
# Ejemplos de uso de las funciones
print('=== Listado de Tablas ===')
tablas = db_tables()
print(f'Tablas en la base de datos: {tablas}')

print('
=== Esquema de la tabla 'usuarios' ===')
esquema = db_table_schema('usuarios')
for col in esquema:
    print(f"Columna: {col['name']}, Tipo: {col['type']}, PK: {col['pk']}")

print('
=== Insertar nuevo usuario ===')
nuevo_usuario = {
    'nombre': 'Ana Martínez',
    'email': 'ana@example.com', 
    'edad': 28
}
resultado_insert = db_insert('usuarios', nuevo_usuario)
print(f'Usuario insertado: {resultado_insert}')

print('
=== Actualizar usuario ===')
resultado_update = db_update('usuarios', 'edad', 31, "nombre = 'Juan Pérez'")
print(f'Usuarios actualizados: {resultado_update}')


## 3. Backend en FastAPI

In [ ]:
from flask import Flask, request, jsonify
import threading
import time
import requests

app = Flask(__name__)

# ------------------------------
# Flask Routes
# ------------------------------
@app.route('/db/update', methods=['POST'])
def route_db_update():
    """
    POST /db/update
    JSON: {'tabla': '...', 'campo': '...', 'valor': <any>, 'condicion_sql': 'id = 1', 'db': '<optional_db_path>'}
    """
    payload = request.get_json(silent=True) or {}
    try:
        tabla = payload['tabla']
        campo = payload['campo']
        valor = payload['valor']
        condicion_sql = payload['condicion_sql']
        data_db = payload.get('db', SQLITE_PATH)
    except KeyError as ke:
        return jsonify({'error': f'Falta el campo requerido: {ke}'}) , 400
    try:
        result = db_update(tabla, campo, valor, condicion_sql, data_db=data_db)
        return jsonify(result)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/db/insert', methods=['POST'])
def route_db_insert():
    """
    POST /db/insert
    JSON: {'tabla': '...', 'valores': {'col1': v1, 'col2': v2, ...}, 'db': '<optional_db_path>'}
    """
    payload = request.get_json(silent=True) or {}
    try:
        tabla = payload['tabla']
        valores = payload['valores']
        data_db = payload.get('db', SQLITE_PATH)
    except KeyError as ke:
        return jsonify({'error': f'Falta el campo requerido: {ke}'}) , 400
    try:
        result = db_insert(tabla, valores, data_db=data_db)
        return jsonify(result)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/db/delete', methods=['POST'])
def route_db_delete():
    """
    POST /db/delete
    JSON: {'tabla': '...', 'condicion_sql': 'id > 5', 'db': '<optional_db_path>'}
    """
    payload = request.get_json(silent=True) or {}
    try:
        tabla = payload['tabla']
        condicion_sql = payload['condicion_sql']
        data_db = payload.get('db', SQLITE_PATH)
    except KeyError as ke:
        return jsonify({'error': f'Falta el campo requerido: {ke}'}) , 400
    try:
        result = db_delete(tabla, condicion_sql, data_db=data_db)
        return jsonify(result)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/db/delete_pk', methods=['POST'])
def route_db_delete_pk():
    """
    POST /db/delete_pk
    JSON: {'tabla': '...', 'pk': '...', 'valor': <any>, 'db': '<optional_db_path>'}
    """
    payload = request.get_json(silent=True) or {}
    try:
        tabla = payload['tabla']
        pk = payload['pk']
        valor = payload['valor']
        data_db = payload.get('db', SQLITE_PATH)
    except KeyError as ke:
        return jsonify({'error': f'Falta el campo requerido: {ke}'}) , 400
    try:
        result = db_delete_pk(tabla, pk, valor, data_db=data_db)
        return jsonify(result)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Listado de tablas
@app.route('/db/tables', methods=['GET'])
def route_db_tables():
    """
    GET /db/tables?db=<optional_db_path>
    Devuelve la lista de tablas de la base de datos.
    """
    data_db = request.args.get('db', SQLITE_PATH)
    try:
        tables = db_tables(data_db=data_db)
        return jsonify({'tables': tables, 'count': len(tables)})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Esquema de una tabla
@app.route('/db/table_schema', methods=['GET'])
def route_db_table_schema():
    """
    GET /db/table_schema?tabla=<nombre>&db=<optional_db_path>
    Devuelve el esquema (columnas) de la tabla indicada.
    """
    tabla = request.args.get('tabla')
    data_db = request.args.get('db', SQLITE_PATH)
    if not tabla:
        return jsonify({'error': "Falta el parámetro requerido: 'tabla'."}), 400
    try:
        validate_identifier(tabla, 'table name')
        schema = db_table_schema(tabla, data_db=data_db)
        return jsonify({'tabla': tabla, 'schema': schema, 'count': len(schema)})
    except ValueError as ve:
        return jsonify({'error': str(ve)}), 400
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Health check
@app.route('/health', methods=['GET'])
def health():
    try:
        conn = get_db_connection(SQLITE_PATH)
        conn.close()
        return jsonify({'status': 'ok', 'backend': DB_BACKEND, 'db': SQLITE_PATH})
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

def run_flask_app():
    """Función para ejecutar Flask en segundo plano"""
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

print('Backend Flask definido correctamente!')


## 4. Ejecutar Servidor en Segundo Plano

In [ ]:
# Ejecutar el servidor Flask en segundo plano
flask_thread = threading.Thread(target=run_flask_app, daemon=True)
flask_thread.start()

# Esperar a que el servidor se inicie
time.sleep(3)

print('Servidor Flask ejecutándose en http://localhost:5000')
print('Endpoints disponibles:')
print('  GET  /health')
print('  GET  /db/tables')
print('  GET  /db/table_schema?tabla=<nombre>')
print('  POST /db/insert')
print('  POST /db/update') 
print('  POST /db/delete')
print('  POST /db/delete_pk')


## 5. Prueba del Servidor desde el Notebook

In [ ]:
# Pruebas de los endpoints del servidor
base_url = 'http://localhost:5000'

print('=== Health Check ===')
try:
    response = requests.get(f'{base_url}/health')
    print(f'Status: {response.status_code}')
    print(f'Response: {response.json()}')
except Exception as e:
    print(f'Error: {e}')

print('
=== Listar Tablas ===')
try:
    response = requests.get(f'{base_url}/db/tables')
    print(f'Status: {response.status_code}')
    print(f'Tablas: {response.json()}')
except Exception as e:
    print(f'Error: {e}')


In [ ]:
print('=== Obtener Esquema de 'usuarios' ===')
try:
    response = requests.get(f'{base_url}/db/table_schema?tabla=usuarios')
    print(f'Status: {response.status_code}')
    print(f'Esquema: {response.json()}')
except Exception as e:
    print(f'Error: {e}')

print('
=== Insertar Nuevo Usuario ===')
nuevo_usuario = {
    'tabla': 'usuarios',
    'valores': {
        'nombre': 'Pedro Sánchez',
        'email': 'pedro@example.com',
        'edad': 40
    }
}
try:
    response = requests.post(f'{base_url}/db/insert', json=nuevo_usuario)
    print(f'Status: {response.status_code}')
    print(f'Resultado: {response.json()}')
except Exception as e:
    print(f'Error: {e}')


In [ ]:
print('=== Actualizar Usuario ===')
actualizacion = {
    'tabla': 'usuarios',
    'campo': 'edad',
    'valor': 32,
    'condicion_sql': "nombre = 'Juan Pérez'"
}
try:
    response = requests.post(f'{base_url}/db/update', json=actualizacion)
    print(f'Status: {response.status_code}')
    print(f'Resultado: {response.json()}')
except Exception as e:
    print(f'Error: {e}')

print('
=== Eliminar Usuario por PK ===')
eliminacion = {
    'tabla': 'usuarios',
    'pk': 'id',
    'valor': 1
}
try:
    response = requests.post(f'{base_url}/db/delete_pk', json=eliminacion)
    print(f'Status: {response.status_code}')
    print(f'Resultado: {response.json()}')
except Exception as e:
    print(f'Error: {e}')


### Resumen de Pruebas Completadas

El servidor Flask está funcionando correctamente y se han probado los siguientes endpoints:

1. ✅ **Health Check** - Verifica que el servidor esté activo
2. ✅ **Listar Tablas** - Obtiene todas las tablas de la base de datos
3. ✅ **Obtener Esquema** - Muestra la estructura de una tabla específica
4. ✅ **Insertar Datos** - Agrega nuevos registros a la base de datos
5. ✅ **Actualizar Datos** - Modifica registros existentes
6. ✅ **Eliminar por PK** - Borra registros usando la clave primaria

El API está listo para ser usado desde cualquier cliente HTTP.
